In [1]:
import torch
import numpy as np
import random
import torch.optim as optim

from algorithms import *
from models import *
from dataloaders import *

from tabulate import tabulate

In [2]:
device = 'cpu'
dataset = AdultDataset(device=device)

In [3]:
def lr_kde_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(seeds[i])
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = logReg(num_features=dataset.XZ_train.shape[1])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_kde_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'], 
            h=hp['h'], 
            delta_huber=hp['delta_huber'], 
            optimal_effort=True, 
            delta_effort=hp['delta_effort']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test


def experiment_runner(dataset, SGD_hp, EI_hp, DP_hp, EO_hp, EODD_hp, seeds):
    
    _, _, SGD = lr_kde_model_runner(dataset, SGD_hp, seeds)
    _, _, EI = lr_kde_model_runner(dataset, EI_hp, seeds)
    _, _, DP = lr_kde_model_runner(dataset, DP_hp, seeds)
    _, _, EO = lr_kde_model_runner(dataset, EO_hp, seeds)
    _, _, EODD = lr_kde_model_runner(dataset, EODD_hp, seeds)
    
    return SGD, EI, DP, EO, EODD

def hyperparameter_test(dataset, hp_test, seed=0):
    hp = hp_test.copy()
    result = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            c = []
            hp['learning_rate'] = i
            hp['lambda_'] = k
            train, val, _ = lr_kde_model_runner(dataset, hp, seeds=[seed])
            c.append(hp['learning_rate'])
            c.append(hp['lambda_'])
            c.append(train['accuracy_mean'])
            c.append(val['accuracy_mean'])
            c.append(val['ei_mean'])
            c.append(val['dp_mean'])
            c.append(val['eo_mean'])
            c.append(val['eodd_mean'])
            result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))

#### SGD Hyperparameter Selection

In [4]:
SGD_hp_test = {}
SGD_hp_test['learning_rate'] = [0.0001, 0.001, 0.01, 0.1]
SGD_hp_test['lambda_'] = [0]
SGD_hp_test['n_epochs'] = 100
SGD_hp_test['batch_size'] = 1024 
SGD_hp_test['fairness'] = ''
SGD_hp_test['h'] = 0.01
SGD_hp_test['delta_huber'] = 0.5
SGD_hp_test['delta_effort'] = 2.1

hyperparameter_test(dataset, SGD_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:17<00:00,  5.74epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:16<00:00,  5.94epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:16<00:00,  6.01epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:16<00:00,  6.04epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val           ei          dp          eo        eodd
---------------  ---------  ----------------  --------------  -----------  ----------  ----------  ----------
         0.0001          0          0.759625        0.759158  0.000638715  0.00732228  0.00195639  0.00195639
         0.001           0          0.837043        0.842533  0.0163369    0.119958    0.0720145   0.0720145
         0.01            0          0.846036        0.85082   0.0588518    0.14373     0.0879802   0.0879802
         0.1             0          0.844875        0.843362  0.0502871    0.102196    0.0600302   0.0600302


The learning rate is decided as 0.01.


In [16]:
SGD_hp = SGD_hp_test.copy()
SGD_hp['learning_rate'] = 0.01
SGD_hp['lambda_'] = 0


#### EI Hyperparameter Test

In [7]:
EI_hp_test = SGD_hp_test.copy()
EI_hp_test['learning_rate'] = [0.01]
EI_hp_test['lambda_'] = [0, 0.2, 0.6, 0.8, 0.9, 0.95]
EI_hp_test['fairness'] = 'EI'

hyperparameter_test(dataset, EI_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:13<00:00,  7.46epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:13<00:00,  7.53epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:13<00:00,  7.45epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:13<00:00,  7.37epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val          ei         dp          eo       eodd
---------------  ---------  ----------------  --------------  ----------  ---------  ----------  ---------
           0.01       0             0.846036        0.85082   0.0588518   0.14373    0.0879802   0.0879802
           0.01       0.2           0.845663        0.849826  0.0537019   0.138236   0.0737163   0.0737163
           0.01       0.6           0.845041        0.850489  0.0414804   0.107657   0.00622383  0.0411582
           0.01       0.8           0.83696         0.840046  0.0339261   0.0610633  0.101126    0.101126
           0.01       0.9           0.820672        0.82662   0.00933132  0.0300189  0.179762    0.179762
           0.01       0.95          0.820755        0.828278  0.0072032   0.0687697  0.0225986   0.0230697


The lambda for EI is decided as 0.9.

In [8]:
EI_hp = EI_hp_test.copy()
EI_hp['learning_rate'] = 0.01
EI_hp['lambda_'] = 0.9

#### DP Hyperparameter Selection

In [9]:
DP_hp_test = SGD_hp_test.copy()
DP_hp_test['learning_rate'] = [0.01]
DP_hp_test['lambda_'] = [0, 0.2, 0.6, 0.8, 0.9, 0.95]
DP_hp_test['fairness'] = 'DP'

hyperparameter_test(dataset, DP_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:11<00:00,  8.71epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:11<00:00,  8.76epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:11<00:00,  8.71epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:11<00:00,  8.79epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei         dp         eo       eodd
---------------  ---------  ----------------  --------------  ---------  ---------  ---------  ---------
           0.01       0             0.846036        0.85082   0.0588518  0.14373    0.0879802  0.0879802
           0.01       0.2           0.845746        0.849329  0.0570534  0.131549   0.053486   0.056273
           0.01       0.6           0.844088        0.847008  0.0484778  0.0898855  0.0536491  0.0536491
           0.01       0.8           0.839322        0.838223  0.0547606  0.0518602  0.161119   0.161119
           0.01       0.9           0.82892         0.834908  0.0396362  0.0276117  0.210223   0.210223
           0.01       0.95          0.822289        0.830764  0.0285062  0.0151801  0.239091   0.239091


The lambda for DP is decided as 0.95.

In [10]:
DP_hp = DP_hp_test.copy()
DP_hp['learning_rate'] = 0.01
DP_hp['lambda_'] = 0.95

#### EO Hyperparameter Selection

In [11]:
EO_hp_test = SGD_hp_test.copy()
EO_hp_test['learning_rate'] = [0.01]
EO_hp_test['lambda_'] = [0, 0.2, 0.6, 0.8, 0.9, 0.95]
EO_hp_test['fairness'] = 'EO'

hyperparameter_test(dataset, EO_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:11<00:00,  8.56epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:11<00:00,  8.67epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:11<00:00,  8.60epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:11<00:00,  8.58epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei        dp          eo       eodd
---------------  ---------  ----------------  --------------  ---------  --------  ----------  ---------
           0.01       0             0.846036        0.85082   0.0588518  0.14373   0.0879802   0.0879802
           0.01       0.2           0.844337        0.851318  0.0651668  0.137198  0.0491986   0.0593867
           0.01       0.6           0.843632        0.849329  0.0489743  0.10621   0.00920131  0.0391501
           0.01       0.8           0.844958        0.852147  0.0511064  0.107653  0.00753381  0.0385376
           0.01       0.9           0.837581        0.845019  0.0257249  0.111527  0.0181395   0.0450961
           0.01       0.95          0.832235        0.831096  0.0467608  0.131444  0.0194323   0.0655647


The lambda for EO is decided as 0.8.

In [12]:
EO_hp = EO_hp_test.copy()
EO_hp['learning_rate'] = 0.01
EO_hp['lambda_'] = 0.8

#### EODD Hyperparameter Selection

In [13]:
EODD_hp_test = SGD_hp_test.copy()
EODD_hp_test['learning_rate'] = [0.01]
EODD_hp_test['lambda_'] = [0, 0.2, 0.6, 0.8, 0.9, 0.95]
EODD_hp_test['fairness'] = 'EODD'

hyperparameter_test(dataset, EODD_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:13<00:00,  7.51epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:13<00:00,  7.50epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:13<00:00,  7.31epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:13<00:00,  7.32epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei         dp          eo       eodd
---------------  ---------  ----------------  --------------  ---------  ---------  ----------  ---------
           0.01       0             0.846036        0.85082   0.0588518  0.14373    0.0879802   0.0879802
           0.01       0.2           0.844627        0.850489  0.0639913  0.136212   0.0488639   0.0589448
           0.01       0.6           0.84529         0.849826  0.0489154  0.104766   0.0128538   0.038698
           0.01       0.8           0.841519        0.846677  0.0431721  0.089058   0.0152306   0.0302909
           0.01       0.9           0.841684        0.845848  0.037819   0.0934981  0.0171985   0.032491
           0.01       0.95          0.823408        0.829438  0.0249595  0.0637278  0.00921847  0.0148947


The lambda for EODD is decided as 0.8.

In [14]:
EODD_hp = EODD_hp_test.copy()
EODD_hp['learning_rate'] = 0.01
EODD_hp['lambda_'] = 0.8

#### Model training

In [17]:
seeds = np.arange(1,6)

SGD, EI, DP, EO, EODD = experiment_runner(dataset, SGD_hp, EI_hp, DP_hp, EO_hp, EODD_hp, seeds)

/var/folders/mt/mgpv1mld71dfr_kn_fq57str0000gn/T/ipykernel_5072/3570034255.py:29: DeprecationWarning: Seeding based on hashing is deprecated
since Python 3.9 and will be removed in a subsequent version. The only 
supported seed types are: None, int, float, str, bytes, and bytearray.
  random.seed(seeds[i])


training seed 1 started


Training: 100%|██████████| 100/100 [00:09<00:00, 10.16epochs/s]


training seed 2 started


Training: 100%|██████████| 100/100 [00:09<00:00, 10.19epochs/s]


...

training seed 4 started


Training: 100%|██████████| 100/100 [00:13<00:00,  7.41epochs/s]


training seed 5 started


Training: 100%|██████████| 100/100 [00:13<00:00,  7.42epochs/s]

Training finished for all seeds.


In [18]:
result = []
models = ["SGD", "EI", "DP", "EO", "EODD"]
sol = [SGD, EI, DP, EO, EODD]
for i in range(len(models)):
    c = []
    c.append(models[i])
    res = sol[i]
    c.append(res['accuracy_mean'])
    c.append(res['accuracy_var'])
    c.append(res['ei_mean'])
    c.append(res['ei_var'])
    c.append(res['dp_mean'])
    c.append(res['dp_var'])
    c.append(res['eo_mean'])
    c.append(res['eo_var'])
    c.append(res['eodd_mean'])
    c.append(res['eodd_var'])
    result.append(c)

print(tabulate(result, headers=["model","accuracy_mean","accuracy_var","ei_mean","ei_var", "dp_mean", "dp_var","eo_mean","eo_var","eodd_mean","eodd_var"]))

model      accuracy_mean    accuracy_var    ei_mean      ei_var    dp_mean      dp_var     eo_mean      eo_var    eodd_mean    eodd_var
-------  ---------------  --------------  ---------  ----------  ---------  ----------  ----------  ----------  -----------  ----------
SGD             0.847968     0.000228481  0.0512867  0.00207924  0.122741   0.00298508  0.0968474   0.00511557    0.0968474  0.00511557
EI              0.825657     0.0016279    0.0147183  0.00437787  0.0237718  0.00391655  0.195367    0.00746519    0.195367   0.00746519
DP              0.824449     0.00502255   0.0278241  0.0104796   0.0127825  0.00683465  0.232965    0.0255199     0.232965   0.0255199
EO              0.845166     0.000952479  0.0483146  0.0042991   0.108956   0.0104912   0.0168327   0.0132126     0.0414505  0.00664543
EODD            0.840412     0.00660143   0.0377342  0.00526133  0.0854877  0.0164578   0.00801349  0.00789422    0.0278046  0.0090731
